# Time Series Exercises

## Data Acquisition

NOTE (Python methods):

- 'append' --> appends object at the end.

- 'extend' --> extends list by appending elements from the iterable.

https://docs.python.org/3/library/urllib.parse.html

### Imports

In [6]:
# Import functions

import pandas as pd
import requests
import os
from acquire import get_items_data_from_api, get_stores_data_from_api
from acquire import get_sales_data_from_api, get_opsd_data,get_combined_item_stores_sales_data

#### (1). Using the code from the lesson as a guide and the REST API from https://python.zgulde.net/api/v1/items as we did in the lesson, create a dataframe named items that has all of the data for items.

### Get Items Data

In [5]:
# Get items data

items = get_items_data_from_api()
items.head()

Getting page 3 of 3: https://python.zgulde.net//api/v1/items?page=3

,item_brand,item_id,item_name,item_price,item_upc12,item_upc14
0,Riceland,1,Riceland American Jazmine Rice,0.84,35200264013,35200264013
1,Caress,2,Caress Velvet Bliss Ultra Silkening Beauty Bar...,6.44,11111065925,11111065925
2,Earths Best,3,Earths Best Organic Fruit Yogurt Smoothie Mixe...,2.43,23923330139,23923330139
3,Boars Head,4,Boars Head Sliced White American Cheese - 120 Ct,3.14,208528800007,208528800007
4,Back To Nature,5,Back To Nature Gluten Free White Cheddar Rice ...,2.61,759283100036,759283100036


In [3]:
items.shape

(50, 6)

#### (2). Do the same thing, but for stores (https://python.zgulde.net/api/v1/stores)

### Stores Data

In [9]:
stores = get_stores_data_from_api()
stores.head()

,store_address,store_city,store_id,store_state,store_zipcode
0,12125 Alamo Ranch Pkwy,San Antonio,1,TX,78253
1,9255 FM 471 West,San Antonio,2,TX,78251
2,2118 Fredericksburg Rdj,San Antonio,3,TX,78201
3,516 S Flores St,San Antonio,4,TX,78204
4,1520 Austin Hwy,San Antonio,5,TX,78218


In [6]:
stores.shape

(10, 5)

#### (3). Extract the data for sales (https://python.zgulde.net/api/v1/sales). There are a lot of pages of data here, so your code will need to be a little more complex. Your code should continue fetching data from the next page until all of the data is extracted.

In [2]:
sales = get_sales_data_from_api()

Fetching page 183/183 https://api.data.codeup.com/api/v1/sales?page=183

In [3]:
sales.head()

,item,sale_amount,sale_date,sale_id,store
0,1,13.0,"Tue, 01 Jan 2013 00:00:00 GMT",1,1
1,1,11.0,"Wed, 02 Jan 2013 00:00:00 GMT",2,1
2,1,14.0,"Thu, 03 Jan 2013 00:00:00 GMT",3,1
3,1,13.0,"Fri, 04 Jan 2013 00:00:00 GMT",4,1
4,1,10.0,"Sat, 05 Jan 2013 00:00:00 GMT",5,1


#### (4). Save the data in your files to local csv files so that it will be faster to access in the future.

In [10]:
items.to_csv('items.csv')
stores.to_csv('stores.csv')
sales.to_csv('sales.csv')

#### (5). Combine the data from your three separate dataframes into one large dataframe.



In [54]:
sales = pd.read_csv('sales.csv')  
sales = sales.reset_index() 
sales = sales.drop(['index'], axis = 1)
sales = sales.drop(columns= 'Unnamed: 0')

# Rename stores columns
sales = sales.rename(columns = {'item':'item_id', 'store':'store_id'})

sales.head(2)

,item_id,sale_amount,sale_date,sale_id,store_id
0,1,13.0,"Tue, 01 Jan 2013 00:00:00 GMT",1,1
1,1,11.0,"Wed, 02 Jan 2013 00:00:00 GMT",2,1


In [55]:
items = pd.read_csv('items.csv')
items = items.drop(columns = 'Unnamed: 0')

items.head(2)

,item_brand,item_id,item_name,item_price,item_upc12,item_upc14
0,Riceland,1,Riceland American Jazmine Rice,0.84,35200264013,35200264013
1,Caress,2,Caress Velvet Bliss Ultra Silkening Beauty Bar...,6.44,11111065925,11111065925


In [56]:
stores = pd.read_csv('stores.csv', index_col = False)
stores = stores.drop(columns = 'Unnamed: 0')
stores.head(2)

,store_address,store_city,store_id,store_state,store_zipcode
0,12125 Alamo Ranch Pkwy,San Antonio,1,TX,78253
1,9255 FM 471 West,San Antonio,2,TX,78251


In [58]:
sales_items = pd.merge(sales, items, how = 'left', on = 'item_id')
merged_df = pd.merge(sales_items, stores, how='left', on='store_id')

In [61]:
merged_df.head(3)

,item_id,sale_amount,sale_date,sale_id,store_id,item_brand,item_name,item_price,item_upc12,item_upc14,store_address,store_city,store_state,store_zipcode
0,1,13.0,"Tue, 01 Jan 2013 00:00:00 GMT",1,1,Riceland,Riceland American Jazmine Rice,0.84,35200264013,35200264013,12125 Alamo Ranch Pkwy,San Antonio,TX,78253
1,1,11.0,"Wed, 02 Jan 2013 00:00:00 GMT",2,1,Riceland,Riceland American Jazmine Rice,0.84,35200264013,35200264013,12125 Alamo Ranch Pkwy,San Antonio,TX,78253
2,1,14.0,"Thu, 03 Jan 2013 00:00:00 GMT",3,1,Riceland,Riceland American Jazmine Rice,0.84,35200264013,35200264013,12125 Alamo Ranch Pkwy,San Antonio,TX,78253


#### (6). Acquire the Open Power Systems Data for Germany, which has been rapidly expanding its renewable energy production in recent years. The data set includes country-wide totals of electricity consumption, wind power production, and solar power production for 2006-2017. You can get the data here: https://raw.githubusercontent.com/jenfly/opsd/master/opsd_germany_daily.csv

In [3]:
open_power_systems  = get_opsd_data()
open_power_systems.head(3)

,Date,Consumption,Wind,Solar,Wind+Solar
0,2006-01-01,1069.184,NaN,NaN,NaN
1,2006-01-02,1380.521,NaN,NaN,NaN
2,2006-01-03,1442.533,NaN,NaN,NaN


#### (7). Make sure all the work that you have done above is reproducible. That is, you should put the code above into separate functions in the acquire.py file and be able to re-run the functions and get the same data.

In [7]:
sales_items_stores = get_combined_item_stores_sales_data()

Getting page 3 of 3: https://python.zgulde.net//api/v1/items?page=3=183